In [98]:
import pandas as pd
import numpy as np
import os
import math

from google.colab import drive
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, HoverTool, WheelZoomTool, Button, TextInput, CustomJS, CheckboxButtonGroup
from bokeh.embed import file_html
from bokeh.layouts import row, gridplot
from bokeh.palettes import Spectral11 as palette
from bokeh.io import curdoc, push_notebook

drive.mount('/content/drive')

output_notebook()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 함수 설명
1. find_file_and_get_raw_datas
  : 지정한 폴더에 지정한 파일 확장자에 해당하는 파일 데이터 프레임을 반환합니다.
  *   매개변수 : folder_path, file_extension 폴더 경로 (문자열)
  *   return : 해당 폴더 안 지정한 파일 확장자 데이터 프레임을 반환

  *   1. 폴더 경로 안 지정한 파일 확장자 파일을 all_file_list_에 저장
  *   2. all_file_list 정렬
  *   3. all_file_list를 통해 해당 파일 정보를 log_datas에 저장 ( * log_datas의 경우 df를 저장한 list 형태)
  * 4. log_datas 반환

  



In [99]:
def find_file_and_get_raw_datas(folder_path, file_extension):
  # 폴더 안 파일을 file_list에 저장
  file_list = os.listdir(folder_path)

  # file_list 중 file_extension으로 지정해준 확장자인 것만 all_file_list에 저장
  all_file_list = []
  for file in file_list:
    if file.endswith(file_extension):
      all_file_list.append(file)

  # all_file_list 정렬
  all_file_list = sorted(all_file_list)
  
  # all_file_list로 해당 파일 정보 df 형태로 가져온 후 리스트에 저장해 반환
  log_datas = []
  for file in all_file_list:
    log_datas.append(pd.read_csv(folder_path + "/" + file, sep=',', encoding="cp949"))
  return log_datas

# 함수 설명
1. make_graphs
  : bokeh library를 이용한 그래프를 작성합니다.





  *   매개변수 :log_datas, y_axis, x_axis_ title, x_axis_label, y_axis_label, s ize, alpha  


---


log_datas : 여러 df가 저장되어 있는 리스트  
y_axis : 출력할 y축 column 이름(String 형태)이 저장되어 있는 리스트  
x_axis : 출력할 x축 column 이름(String 형태)을 지정  

( y_axis는 여러개를 지정할 수 있지만, x_axis는 한개만 지정 가능  
ex) y_axis = ["singular_value", "manipulability", "joint_vel4"]

title : 그래프의 title 설정  
x_axis_label : 그래프의 x 축 이름 설정  
y_axis_label : 그래프의 y 축 이름 설정  

size : 그래프 원의 크기 조정  
alpha : 그래프 원의 투명도 조정  

ex) plots = make_graphs(df_datas, ["singular_value", "manipulability", "joint_vel4"], "joint_pos_ref5", "test", "joint_pos_ref5", "SVD")  

---


  
  *   return : 해당 폴더 안 지정한 파일 확장자 파일 리스트를 반환

  *   1. 폴더 경로 안 지정한 파일 확장자 파일을 all_file_list_에 저장
  *   2. all_file_list 정렬
  *   3. all_file_list를 통해 해당 파일 정보를 log_datas에 저장 ( * log_datas의 경우 df를 저장한 list 형태)
  * 4. log_datas 반환

!! 주의 from bokeh.palettes import Spectral11 as palette  
11가지 색만 지원하기 때문에 출력할 y축의 개수가 11개가 넘어갈 경우 팔레트 범위 오류 발생
  



In [100]:
def make_graphs(log_datas, y_axis, x_axis, title, x_axis_label, y_axis_label, size, alpha):
  plots = []
  # palette 최대 갯수
  max_color = 10

  if len(y_axis) > max_color:
    print("y축은 최대 11개 까지 가능합니다.")
    return None

  for i in range(len(log_datas)):
    curdoc().theme = "caliber"
    TOOLS = "pan,wheel_zoom,box_select,lasso_select,reset, save, hover"
    TOOLTIPS = [
      ("index", "$x"),
      ("y_value", "$y)"),
    ]

    p = figure(title=title, x_axis_label=x_axis_label, y_axis_label=y_axis_label, plot_width=1200, plot_height=900, tools=TOOLS, tooltips=TOOLTIPS)

    for j in range(0, len(y_axis)):
      # 색깔 지정을 위한 if, else
      if j == 0:
        p.extra_y_ranges = {y_axis[j] : Range1d(start=min(log_datas[i][y_axis[j]]), end=max(log_datas[i][y_axis[j]]))}
        p.circle(log_datas[i][x_axis], log_datas[i][y_axis[j]], y_range_name=y_axis[j], line_color=palette[j], size=size, alpha=alpha)

      else:
        p.extra_y_ranges[y_axis[j]] = Range1d(start=min(log_datas[i][y_axis[j]]), end=max(log_datas[i][y_axis[j]]))
        if j % 2 == 0:
          p.circle(log_datas[i][x_axis], log_datas[i][y_axis[j]], y_range_name=y_axis[j], line_color=palette[j], size=size, alpha=alpha)
          p.add_layout(LinearAxis(y_range_name=y_axis[j], axis_label=y_axis[j], axis_label_text_color=palette[j], axis_line_color=palette[j]), "right")
        else :
          p.circle(log_datas[i][x_axis], log_datas[i][y_axis[j]], y_range_name=y_axis[j], line_color=palette[-j], size=size, alpha=alpha)
          p.add_layout(LinearAxis(y_range_name=y_axis[j], axis_label=y_axis[j], axis_label_text_color=palette[-j], axis_line_color=palette[j]), "right")

    p.legend.orientation = "horizontal"
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.background_fill_color="#f0f0f0"
    p.grid.grid_line_color="#f5f5f5"
    p.toolbar.active_scroll = p.select_one(WheelZoomTool)

    plots.append(p)
  return plots

In [101]:
def start_making_graph():
  # 1. find_file_and_get_raw_datas 함수를 이용해
  # 폴더 안 csv 파일 가져오기
  df_datas = find_file_and_get_raw_datas("/content/drive/MyDrive/Robostar/W8", ".csv")
  # 순차 정보를 위한 df에 index 추가
  for i in range(0, len(df_datas)):
    index = [x for x in range(1, len(df_datas[i]) + 1)]
    df_datas[i]["index"] = index
  # df 리스트 확인
  print(len(df_datas))

  ###########################################################################################################################################################################
  print("============================== SVD, Manip, Velocity4 ==============================")
  # singular_value, manipulability, joint_vel4를 이용한 그래프 그리기
  plots = make_graphs(df_datas, ["singular_value", "manipulability", "joint_vel4"], "joint_pos_ref5", "SVD&mManip&Vel4", "joint_pos_ref5", "SVD", 3, 0.4)
  plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
  # output_file(folder_path + "/" + "SVD&Manip&Vel4" + '.html')
  show(plot, new="tab")

  ###########################################################################################################################################################################
  print("============================== Velocity 1-6 ==============================")
  plots = make_graphs(df_datas, ["joint_vel1", "joint_vel2", "joint_vel3", "joint_vel4", "joint_vel5", "joint_vel6"], "index", "Velocity1-6", "index", "Velocity1-6", 3, 0.4)
  plot = gridplot(plots, ncols=1, toolbar_location="right", plot_width=1200, plot_height=600)
  # output_file(folder_path + "/" + "SVD&Manip&Vel4" + '.html')
  show(plot, new="window")

In [102]:
start_making_graph()

Output hidden; open in https://colab.research.google.com to view.